In [ ]:
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [ ]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

the data set that we are going to use CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on four types of named entities: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups.

In [ ]:
conll2003 = datasets.load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

# **Now we will explore the conll2003 data set**

In [ ]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
conll2003["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [ ]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
conll2003["train"].features["tokens"]

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

In [ ]:
conll2003["train"].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
conll2003["train"].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
example_text = conll2003["train"][0]

In [ ]:
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
example_text["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [ ]:
tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

In [ ]:
tokenized_input["input_ids"]

[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]

In [ ]:
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [ ]:
tokens

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [ ]:
word_ids = tokenized_input.word_ids()

print(word_ids)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]


In [ ]:
example_text["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [ ]:
for i, label in enumerate(example_text["ner_tags"]):
  print(i,label)

0 3
1 0
2 7
3 0
4 0
5 0
6 7
7 0
8 0


In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):

    #tokeinze ids
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []


    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.

        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
q=tokenize_and_align_labels(conll2003["train"][4:5])

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
germany_________________________________ 5
'_______________________________________ 0
s_______________________________________ 0
representative__________________________ 0
to______________________________________ 0
the_____________________________________ 0
european________________________________ 3
union___________________________________ 4
'_______________________________________ 0
s_______________________________________ 0
veterinary______________________________ 0
committee_______________________________ 0
werner__________________________________ 1
z_______________________________________ 2
##wing__________________________________ 2
##mann__________________________________ 2
said____________________________________ 0
on______________________________________ 0
wednesday_______________________________ 0
consumers_______________________________ 0
should__________________________________ 0
buy_____________________________________ 0
sheep___

In [ ]:
## Applying on entire data
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels=9)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=1,
weight_decay=0.01
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install transformers[torch] -q

The DefaultDataCollator in the Hugging Face Transformers library is a utility that batches and pads input sequences to a uniform length, ensuring consistent input size for transformer models. It automatically handles padding, tokenization outputs, and label collation, converting the data into PyTorch or TensorFlow tensors. This simplifies data preparation by automating the process of creating batches with uniform input sizes, making it especially useful when working with variable-length sequences.


In [ ]:
from transformers import DefaultDataCollator

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
)

In [ ]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
example=conll2003['train'][0]

In [ ]:
label_list = conll2003["train"].features["ner_tags"].feature.names

label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
for i in example["ner_tags"]:
  print(i)

3
0
7
0
0
0
7
0
0


In [ ]:
print(example)

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [ ]:
labels = [label_list[i] for i in example["ner_tags"]]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [ ]:
metric.compute(predictions=[labels],references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [ ]:
from transformers import DataCollatorForTokenClassification

In [ ]:
data_collator=DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer=Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.232800,0.068976,0.906432,0.922251,0.914273,0.981127


TrainOutput(global_step=878, training_loss=0.16946385933217806, metrics={'train_runtime': 182.0718, 'train_samples_per_second': 77.118, 'train_steps_per_second': 4.822, 'total_flos': 341387954498718.0, 'train_loss': 0.16946385933217806, 'epoch': 1.0})

In [ ]:
model.save_pretrained("ner_model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}

In [ ]:
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [ ]:
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [ ]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

In [ ]:
import json

In [ ]:
config=json.load(open("/content/ner_model/config.json"))

In [ ]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.44.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [ ]:
config["id2label"] = id2label


In [ ]:
config["label2id"] = label2id

In [ ]:
json.dump(config,open("/content/ner_model/config.json","w"))

In [ ]:
model_fine_tuned=AutoModelForTokenClassification.from_pretrained("ner_model")

In [ ]:
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

# transformer pipeline
Using transformer pipeline to access the model and run it with a few examples!!

In [ ]:
from transformers import pipeline

In [ ]:
nlp_pipeline=pipeline("ner",model=model_fine_tuned,tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
examples = []
answers = []
examples.append("chirag is an intern at mindslab")
examples.append("who is the founder of microsoft is it Josh?")
examples.append("apple launch mobile while eating apple which taste like orange")
examples.append("deven is working ai engineer in google")
examples.append("Microsoft created their software by idea that came from the window of the house")
examples.append("Chirag hopes to become a datascientist one day")

In [ ]:
for example in examples:
  answers.append(nlp_pipeline(example))
  print(example)
  print(answers[-1])

chirag is an intern at mindslab
[{'entity': 'B-PER', 'score': 0.98390514, 'index': 1, 'word': 'chi', 'start': 0, 'end': 3}, {'entity': 'B-PER', 'score': 0.9838912, 'index': 2, 'word': '##rag', 'start': 3, 'end': 6}, {'entity': 'B-ORG', 'score': 0.86317015, 'index': 7, 'word': 'minds', 'start': 23, 'end': 28}, {'entity': 'I-ORG', 'score': 0.561066, 'index': 8, 'word': '##lab', 'start': 28, 'end': 31}]
who is the founder of microsoft is it Josh?
[{'entity': 'B-ORG', 'score': 0.8888511, 'index': 6, 'word': 'microsoft', 'start': 22, 'end': 31}, {'entity': 'B-PER', 'score': 0.9611257, 'index': 9, 'word': 'josh', 'start': 38, 'end': 42}]
apple launch mobile while eating apple which taste like orange
[{'entity': 'B-ORG', 'score': 0.8580454, 'index': 1, 'word': 'apple', 'start': 0, 'end': 5}]
deven is working ai engineer in google
[{'entity': 'B-PER', 'score': 0.9806872, 'index': 1, 'word': 'dev', 'start': 0, 'end': 3}, {'entity': 'B-PER', 'score': 0.9828163, 'index': 2, 'word': '##en', 'start

In [ ]:
answers[-1]

[{'entity': 'B-PER',
  'score': 0.9888868,
  'index': 1,
  'word': 'chi',
  'start': 0,
  'end': 3},
 {'entity': 'B-PER',
  'score': 0.9871279,
  'index': 2,
  'word': '##rag',
  'start': 3,
  'end': 6}]

As we can see we are not getting the Entities as a whole word rather the name Shubham comes out as Shu bham so now what we will do is to make a function that we get us the original words.

In [ ]:
def combine_entities(entities):
    combined_entities = []
    i = 0

    while i < len(entities):
        current_entity = entities[i]
        combined_word = current_entity['word']
        combined_score = current_entity['score']
        start_index = i
        start_index_word = current_entity["start"]
        current_end = current_entity['end']

        # Combine consecutive entities of the same type
        while (i < len(entities) -1 and current_end == entities[i + 1]['start'] and
               (current_entity['entity'][2:] == entities[i + 1]['entity'][2:] or "MISC" in entities[i + 1]['entity'])):

            # Remove '##' from the next word and combine
            combined_word += entities[i + 1]['word'][2:]  # Combine the words
            combined_score += entities[i + 1]['score'] # Sum the scores
            current_end = entities[i + 1]['end']  # Update current_end to the end of the last word
            i += 1  # Move to the next entity
        # Create combined entity with average score
        combined_entity = {
            'entity': current_entity['entity'],  # Keep the type of the first entity
            'score': combined_score / (i - start_index + 1),  # Average score for all combined
            'index': current_entity['index'],  # Keeping the index of the first word
            'word': combined_word,
            'start': start_index_word,
            'end': current_end  # Update to the end of the last combined entity
        }

        combined_entities.append(combined_entity)
        i += 1  # Move to the next entity

    return combined_entities

In [ ]:
example = "chirag is working in a start-up called EvoAstra"
example_ans = nlp_pipeline(example)

In [ ]:
example_ans

[{'entity': 'B-PER',
  'score': 0.9811739,
  'index': 1,
  'word': 'chi',
  'start': 0,
  'end': 3},
 {'entity': 'B-PER',
  'score': 0.9816428,
  'index': 2,
  'word': '##rag',
  'start': 3,
  'end': 6},
 {'entity': 'B-ORG',
  'score': 0.9880752,
  'index': 11,
  'word': 'ev',
  'start': 39,
  'end': 41},
 {'entity': 'B-ORG',
  'score': 0.98173803,
  'index': 12,
  'word': '##oa',
  'start': 41,
  'end': 43},
 {'entity': 'B-ORG',
  'score': 0.9842738,
  'index': 13,
  'word': '##stra',
  'start': 43,
  'end': 47}]

In [ ]:
combined = combine_entities(example_ans)
print(combined)

[{'entity': 'B-PER', 'score': 0.9814083576202393, 'index': 1, 'word': 'chirag', 'start': 0, 'end': 6}, {'entity': 'B-ORG', 'score': 0.9846956729888916, 'index': 11, 'word': 'evoastra', 'start': 39, 'end': 47}]


In [ ]:
answers_combined = []
for answer in answers:
  answers_combined.append(combine_entities(answer))

In [ ]:
for i in answers_combined:
  print(i)

[{'entity': 'B-PER', 'score': 0.9838981628417969, 'index': 1, 'word': 'chirag', 'start': 0, 'end': 6}, {'entity': 'B-ORG', 'score': 0.7121180295944214, 'index': 7, 'word': 'mindslab', 'start': 23, 'end': 31}]
[{'entity': 'B-ORG', 'score': 0.8888511061668396, 'index': 6, 'word': 'microsoft', 'start': 22, 'end': 31}, {'entity': 'B-PER', 'score': 0.9611256718635559, 'index': 9, 'word': 'josh', 'start': 38, 'end': 42}]
[{'entity': 'B-ORG', 'score': 0.8580453991889954, 'index': 1, 'word': 'apple', 'start': 0, 'end': 5}]
[{'entity': 'B-PER', 'score': 0.9817517399787903, 'index': 1, 'word': 'deven', 'start': 0, 'end': 5}, {'entity': 'B-ORG', 'score': 0.9157434105873108, 'index': 8, 'word': 'google', 'start': 32, 'end': 38}]
[{'entity': 'B-ORG', 'score': 0.9662736654281616, 'index': 1, 'word': 'microsoft', 'start': 0, 'end': 9}]
[{'entity': 'B-PER', 'score': 0.9880073070526123, 'index': 1, 'word': 'chirag', 'start': 0, 'end': 6}]


Now we get the NER maping in a better manner. Lets highlight the words of the sentence which are recognized by NER.

In [ ]:
def highlight_text(original_text, entities):
   # map get the enities by the code
    entity_map = {
        'PER': 'Person',
        'ORG': 'Organization',
        'LOC': 'Location',
        'MISC': 'Miscellaneous'
    }

    # A dictionary to hold recognized entities
    recognized_entities = {
        'Person': [],
        'Organization': [],
        'Location': [],
        'Miscellaneous': []
    }

    # Create a list to hold the highlighted text segments
    highlighted_text_segments = []
    last_index = 0  # To keep track of the last highlighted index

    # Iterate through entities to highlight them and build the summary
    for entity in entities:
        # Clean up entity type
        entity_type = entity['entity'][2:] # Get the base type
        start = entity['start']
        end = entity['end']

        # Get the text to highlight using the indices
        highlighted_text_segments.append(original_text[last_index:start])  # Add text before the entity
        word = original_text[start:end]  # Extract the word using indices
        highlighted_text_segments.append(f"[{word}]")  # Highlight the entity
        last_index = end  # Update the last index

        # Map the entity type to the corresponding category
        if entity_type in entity_map:
            category = entity_map[entity_type]
            recognized_entities[category].append(word)

    # Add any remaining text after the last entity
    highlighted_text_segments.append(original_text[last_index:])

    # Combine highlighted text segments
    highlighted_text = ''.join(highlighted_text_segments)

    return {
        'highlighted_text': highlighted_text,
        'recognized_entities': recognized_entities
    }

In [ ]:
example = "chirag is working in a start-up called EvoAstra"
example_ans = nlp_pipeline(example)
combined = combine_entities(example_ans)
print(combined)

[{'entity': 'B-PER', 'score': 0.9814083576202393, 'index': 1, 'word': 'chirag', 'start': 0, 'end': 6}, {'entity': 'B-ORG', 'score': 0.9846956729888916, 'index': 11, 'word': 'evoastra', 'start': 39, 'end': 47}]


In [ ]:
highlight_text(example,combined)

{'highlighted_text': '[chirag] is working in a start-up called [EvoAstra]',
 'recognized_entities': {'Person': ['chirag'],
  'Organization': ['EvoAstra'],
  'Location': [],
  'Miscellaneous': []}}

Now we just need a function to print the dictionary!!

In [ ]:
def print_highlighted_text(original_text, entities):
    # Get the highlighted text and recognized entities
    result = highlight_text(original_text, entities)

    # Print the highlighted text
    print("Highlighted Text:")
    print(result['highlighted_text'])

    # Print recognized entities
    print("\nRecognized Entities:")
    for category, words in result['recognized_entities'].items():
        if words:  # Only print if there are recognized words
            print(f"{category}: {', '.join(words)}")

    print("\n")

In [ ]:
print_highlighted_text(example,combined)

Highlighted Text:
[chirag] is working in a start-up called [EvoAstra]

Recognized Entities:
Person: chirag
Organization: EvoAstra




In [ ]:
for i in range(len(answers_combined)):
  print_highlighted_text(examples[i],answers_combined[i])

Highlighted Text:
[chirag] is an intern at [mindslab]

Recognized Entities:
Person: chirag
Organization: mindslab


Highlighted Text:
who is the founder of [microsoft] is it [Josh]?

Recognized Entities:
Person: Josh
Organization: microsoft


Highlighted Text:
[apple] launch mobile while eating apple which taste like orange

Recognized Entities:
Organization: apple


Highlighted Text:
[deven] is working ai engineer in [google]

Recognized Entities:
Person: deven
Organization: google


Highlighted Text:
[Microsoft] created their software by idea that came from the window of the house

Recognized Entities:
Organization: Microsoft


Highlighted Text:
[Chirag] hopes to become a datascientist one day

Recognized Entities:
Person: Chirag


